In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import io # Input/Output Module
import os # OS interfaces
import cv2 # OpenCV package
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from urllib import request # module for opening HTTP requests
from matplotlib import pyplot as plt # Plotting library

<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=left>
</div>


KUL H02A5a Computer Vision: Group Assignment 1
---------------------------------------------------------------
Student numbers: <span style="color:red">r0846712, r2, r3, r4, r5</span>.

The goal of this assignment is to explore more advanced techniques for constructing features that better describe objects of interest and to perform face recognition using these features. This assignment will be delivered in groups of 5 (either composed by you or randomly assigned by your TA's).

In this assignment you are a group of computer vision experts that have been invited to ECCV 2021 to do a tutorial about  "Feature representations, then and now". To prepare the tutorial you are asked to participate in a kaggle competition and to release a notebook that can be easily studied by the tutorial participants. Your target audience is: (master) students who want to get a first hands-on introduction to the techniques that you apply.

---------------------------------------------------------------
This notebook is structured as follows:
0. Data loading & Preprocessing
1. Feature Representations
2. Evaluation Metrics 
3. Classifiers
4. Experiments
5. Publishing best results
6. Discussion

Make sure that your notebook is **self-contained** and **fully documented**. Walk us through all steps of your code. Treat your notebook as a tutorial for students who need to get a first hands-on introduction to the techniques that you apply. Provide strong arguments for the design choices that you made and what insights you got from your experiments. Make use of the *Group assignment* forum/discussion board on Toledo if you have any questions.

Fill in your student numbers above and get to it! Good luck! 


<div class="alert alert-block alert-info">
<b>NOTE:</b> This notebook is just a example/template, feel free to adjust in any way you please! Just keep things organised and document accordingly!
</div>

<div class="alert alert-block alert-info">
<b>NOTE:</b> Clearly indicate the improvements that you make!!! You can for instance use titles like: <i>3.1. Improvement: Non-linear SVM with RBF Kernel.<i>
</div>
    
---------------------------------------------------------------
# 0. Data loading & Preprocessing

## 0.1. Loading data
The training set is many times smaller than the test set and this might strike you as odd, however, this is close to a real world scenario where your system might be put through daily use! In this session we will try to do the best we can with the data that we've got! 

In [ ]:
# Download data from Kaggle and unzip it
from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile
import os

api = KaggleApi()
api.authenticate()

competition_name = "kul-h02a5a-computervision-groupassignment0"
api.competition_download_files(competition_name,
                               path=os.getcwd())

dataset_name = competition_name + '.zip'
with ZipFile(dataset_name, 'r') as zo:
    zo.extractall()

In [ ]:
# Extract the relevant data for further use (This cell and the one above replace the original cell for reading the data
def readData(filename:str, indexCol:int) -> pd.DataFrame:
    df = pd.read_csv(filename, index_col=indexCol)
    df.index = df.index.rename('id')
    return df

train = readData('train_set.csv', 0)
test = readData('test_set.csv', 0)

In [ ]:
# read the images (cv2.COLOR_BGR2RGB = 4)
import os

# Method 1 for getting the images - using a function
def readImages(data:str, cv_colorspace:int) -> list:
    path = os.path.join(os.getcwd(), data, data)
    images = [cv2.cvtColor(np.load(os.path.join(path, (data + '_' + str(i) + '.npy'))), cv_colorspace) for i in eval('range(eval(data).index.size)')]
    return np.asarray(images, dtype='object')

tr = readImages('train', 4)
te = readImages('test', 4)

# Method 2 for getting the images - without a function
# tr = [cv2.cvtColor(np.load('./train/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) for index, row in train.iterrows()]

# te = [cv2.cvtColor(np.load('./test/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) for index, row in test.iterrows()]
                
print(f"The training set contains {len(train)} examples, the test set contains {len(test)} examples.")

In [ ]:
# Adding the extracted images as a new column to the existing train and test dataframes
# (some issues with chained indexing exist)

# Method 1 - Using insert
# train.insert(1, 'img', tr, True)
# test.insert(2, 'img', tr, True)

# Method 2 - using loc
train.loc[:, 'img'] = tr
test.loc[:, 'img'] = te

In [ ]:
# # Input data files are available in the read-only "../input/" directory

# train = pd.read_csv(
#     '/kaggle/input/kul-h02a5a-computervision-groupassignment0/train_set.csv', index_col = 0)
# train.index = train.index.rename('id')

# test = pd.read_csv(
#     '/kaggle/input/kul-h02a5a-computervision-groupassignment0/test_set.csv', index_col = 0)
# test.index = test.index.rename('id')

# # read the images as numpy arrays and store in "img" column
# train['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computervision-groupassignment0/train/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
#                 for index, row in train.iterrows()]

# test['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computervision-groupassignment0/test/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
#                 for index, row in test.iterrows()]
  

# train_size, test_size = len(train),len(test)

# "The training set contains {} examples, the test set contains {} examples.".format(train_size, test_size)

*Note: this dataset is a subset of the* [*VGG face dataset*](https://www.robots.ox.ac.uk/~vgg/data/vgg_face/).

## 0.2. A first look
Let's have a look at the data columns and class distribution.

In [ ]:
# The training set contains an identifier, name, image information and class label
train.head(1)

In [ ]:
# The test set only contains an identifier and corresponding image information.
test.head(1)

In [ ]:
# The class distribution in the training set:
train.groupby('name').agg({'img':'count', 'class': 'max'})

Note that **Jesse is assigned the classification label 1**, and **Mila is assigned the classification label 2**. The dataset also contains 20 images of **look alikes (assigned classification label 0)** and the raw images. 

## 0.3. Preprocess data
### 0.3.1 Example: HAAR face detector
In this example we use the [HAAR feature based cascade classifiers](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html) to detect faces, then the faces are resized so that they all have the same shape. If there are multiple faces in an image, we only take the first one. 

<div class="alert alert-block alert-info"> <b>NOTE:</b> You can write temporary files to <code>/kaggle/temp/</code> or <code>../../tmp</code>, but they won't be saved outside of the current session
</div>


In [ ]:
class HAARPreprocessor():
    """Preprocessing pipeline built around HAAR feature based cascade classifiers. """
    
    def __init__(self, path, face_size):
        self.face_size = face_size
        file_path = os.path.join(path, "haarcascade_frontalface_default.xml")
        if not os.path.exists(file_path): 
            if not os.path.exists(path):
                os.mkdir(path)
            self.download_model(file_path)
        
        self.classifier = cv2.CascadeClassifier(file_path)
  
    def download_model(self, path):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/data/"\
            "haarcascades/haarcascade_frontalface_default.xml"
        
        with request.urlopen(url) as r, open(path, 'wb') as f:
            f.write(r.read())
            
    def detect_faces(self, img):
        """Detect all faces in an image."""
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return self.classifier.detectMultiScale(
            img_gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
    def extract_faces(self, img):
        """Returns all faces (cropped) in an image."""
        
        faces = self.detect_faces(img)

        return [img[y:y+h, x:x+w] for (x, y, w, h) in faces]
    
    def preprocess(self, data_row):
        faces = self.extract_faces(data_row['img'])
        
        # if no faces were found, return None
        if len(faces) == 0:
            nan_img = np.empty(self.face_size + (3,))
            nan_img[:] = np.nan
            return nan_img
        
        # only return the first face
        return cv2.resize(faces[0], self.face_size, interpolation = cv2.INTER_AREA)
            
    def __call__(self, data):
        return np.stack([self.preprocess(row) for _, row in data.iterrows()]).astype(int)

**Visualise**

Let's plot a few examples.

In [ ]:
# parameter to play with 

# Rajat - Added data type to imshow
FACE_SIZE = (100, 100)

def plot_image_sequence(data, n, imgs_per_row=7):
    n_rows = 1 + int(n/(imgs_per_row+1))
    n_cols = min(imgs_per_row, n)

    f,ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
    for i in range(n):
        if n == 1:
            ax.imshow(data[i].astype('uint8'))
        elif n_rows > 1:
            ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(data[i].astype('uint8'))
        else:
            ax[int(i%n)].imshow(data[i].astype('uint8'))
    plt.show()

    
#preprocessed data 
preprocessor = HAARPreprocessor(path = '../../tmp', face_size=FACE_SIZE)

train_X, train_y = preprocessor(train), train['class'].values
test_X = preprocessor(test)

In [ ]:
# plot faces of Michael and Sarah
plot_image_sequence(train_X[train_y == 0], n=20, imgs_per_row=10)

In [ ]:
# plot faces of Jesse
plot_image_sequence(train_X[train_y == 1], n=30, imgs_per_row=10)

In [ ]:
# plot faces of Mila
plot_image_sequence(train_X[train_y == 2], n=30, imgs_per_row=10)

## 0.4. Store Preprocessed data (optional)
<div class="alert alert-block alert-info">
<b>NOTE:</b> You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All". Feel free to use this to store intermediary results.
</div>

In [ ]:
# save preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
    
# np.save(os.path.join(prep_path, 'train_X.npy'), train_X)
# np.save(os.path.join(prep_path, 'train_y.npy'), train_y)
# np.save(os.path.join(prep_path, 'test_X.npy'), test_X)

# load preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
# train_X = np.load(os.path.join(prep_path, 'train_X.npy'))
# train_y = np.load(os.path.join(prep_path, 'train_y.npy'))
# test_X = np.load(os.path.join(prep_path, 'test_X.npy'))

In [ ]:
# Rajat - saving of data on machine
# save preprocessed data
CURRENT_PATH = os.getcwd()
DATASTORE_PATH = os.path.join(CURRENT_PATH, 'datastore')
if not os.path.exists(DATASTORE_PATH):
    os.mkdir(DATASTORE_PATH)
    
np.save(os.path.join(DATASTORE_PATH, 'train_X.npy'), train_X)
np.save(os.path.join(DATASTORE_PATH, 'train_y.npy'), train_y)
np.save(os.path.join(DATASTORE_PATH, 'test_X.npy'), test_X)

In [ ]:
# Load Data
# load preprocessed data
CURRENT_PATH = os.getcwd()
DATASTORE_PATH = os.path.join(CURRENT_PATH, 'datastore')
train_X = np.load(os.path.join(DATASTORE_PATH, 'train_X.npy'))
train_y = np.load(os.path.join(DATASTORE_PATH, 'train_y.npy'))
test_X = np.load(os.path.join(DATASTORE_PATH, 'test_X.npy'))

Now we are ready to rock!

# 1. Feature Representations
## 1.0. Example: Identify feature extractor
Our example feature extractor doesn't actually do anything... It just returns the input:
$$
\forall x : f(x) = x.
$$

It does make for a good placeholder and baseclass ;).

In [ ]:
class IdentityFeatureExtractor:
    """A simple function that returns the input"""
    
    def transform(self, X):
        return X
    
    def __call__(self, X):
        return self.transform(X)

## 1.1. Baseline 1: HOG feature extractor/Scale Invariant Feature Transform

_**This section deals with the HOG Feature Descriptor. The following are the topics that have been coded and described below. For an in-depth understanding of this topic, sky is the limit. Therefore, only a brief description has been given here. For more specific details, please refer to that particular code where appropriate comments have been given**_ 


* Features
    * Feature Selection
    * Feature Detection
    * Feature Extraction
    * Feature Description
* HOG Feature Descriptor | Hog Feature Extractor
* Scale Invariant Feature Transform (SIFT)
* Feature Matching

*********************************************************************************

### Features

In the context of images, features are essentially low-level characteristics, such as corners, edges, blobs that can be separated out from the images for further processing. The algorithm or the machine learning technique employed can be trained to detect and obtain them automatically. This four steps are generally performed in the same chronological order as they are described here.

Feature selection is the process that focusses on choosing the relevant details in images that are required for further training, prediction. In other words, they make up the areas (2D) or the regions (3D) of interest.

Feature detection is the technique of finding or locating those required features in the image. There are several algorithms that are related to object detection etc. Thus, the primary objective here is to identify those areas and regions of interest in the images.

In order to ensure that the features detected can be used for further image processing and machine learning, it is necessary to convert the extract the detected features and store them in a numerical format that makes it possible to perform mathematical operations on them. 

Feature description (at times used with feature extraction) is the basically the same thing as feature extraction. It comprises of the feature vectors, called as feature descriptors, which represent the features from  the image in a mathematical form.


### Histogram of Oriented Gradients (HOG Feature Extractor/Descriptor) (HOG)

On the basis of the above description regarding the features, HOG is a feature descriptor and/or feature extractor that is widely used for computer vision related tasks and image processing that uses the image gradients (magnitude and direction (orientation). It is primarily employed for detecting objects in the images.

How it achieves this goal can be understood through the underlying fundamental concept that the local features within an image can be illustrated and described through the variation or the distribution of the intensity gradients. Working with not only the magnitude but also the direction makes HOG quite a nice option as a feature descriptor. It works by segmenting the whole image into a number of rectangular cells (a grid) and then for the pixels contained within each cell, a histogram of gradient directions is calculated. The final descriptor is the combination of all such histograms.

HOG descriptor offers some advantages over other descriptors because of the fact that it operates on cells locally and thus is invariant to geometric and photometric transformations other than for the object (in the image to be detected) orientation itself. Thus, in a way, HOG lays emphasis on the structure and object shape. Broadly speaking, the basic steps mentioned below are involved in the calculation of the HOG. This is what happens in the background when the _hog_ function from _sklearn_ is called. Steps like normalization of the block are specified as parameters in the function call

 * Computation of the Image Gradients (in different directions)
 * Orientation Bins: creation of the histograms for each cell)
 * Descriptor Blocks: consists of the local normalization of the gradient strengths
 * Block Normalization: Four types (L1-sqrt, L1-norm, L2-hys, L2-norm)
 * Detection of the object
 
As with most of the computer vision tasks, prior to the calculation of the image gradients in X and Y directions (magnitude and direction) for HOG feature extraction, adequate image pre-processing must be performed. This involves re-sizing of the image etc. 


*********************************************************************************

### Scale Invarient Feature Transform (SIFT) and Feature Matching (Brute Force)

Similar to HOG, SIFT is too a feature detection algorithm in computer vision, used for detecting and describing the local features in an image. It has many applications with one of the them being object recognition. Firstly, it extracts the keypoints (another name for _features_ and thereafter computes their descriptors. As the name says, a big advantage of this algorithm is its ability to be scale invariant which was an imporvement over other feature extractors/descriptors such as the Harris corner detector which were rotation invariant but not scale invariant. SIFT addresses that problem. 

There are mainly four parts to implementing SIFT (using OpenCV):

 * Scale-space Extreme Detection
 * Localization of the keypoints
 * Orientation Assignment
 * Keypoint Descriptor
 * Matching of the Keypoints
 
After the application of SIFT, a brute force matcher can be used which basically takes the descriptor of an image and matches it with the descriptors in the other image. THis matching is in terms of the distance, Euclidean, for example.

Both of these algorithms were coded using openCV and comments at appropriate places in the relevant section below explain more specifically. For more info regarding SIFT, the reader is advised to refer to the original paper by D.Lowe, which talks about SIFT in more detail (_Distinctive Image Features from Scale-Invariant Keypoints_).

Also the t-SNE plot which visualises data in higher dimensions in a lower dimension (2 or 3) using a coordinate (location) for each datapoint using a statistical method.

In [ ]:
# class HOGFeatureExtractor(IdentityFeatureExtractor):
#     """TODO: this feature extractor is under construction"""
    
#     def __init__(**params):
#         self.params = params
        
#     def transform(self, X):
#         raise NotImplmentedError

In [ ]:
# HOG Feature Extractor
from skimage.feature import hog
from skimage.transform import resize

# Function for finding the HOG Features/Descriptors
def hogFeature(image):
    fd, hogImage = hog(image, orientations=9, 
                       pixels_per_cell=(8, 8), 
                       cells_per_block=(2, 2), 
                       visualize=True, 
                       multichannel=True,
                       block_norm='L2')
    
    return fd, hogImage

# Calling the HOG Function from above on the Training Dataset
# fds consists of the Feature Descriptors for all the images in the Training Dataset
# hogs comprises of the HOG images for all the images in the Training Dataset

fds, hogs = [], []
for i in range(len(train_X)):
    feature, hogImage = hogFeature(train_X[i])
    fds.append(feature)
    hogs.append(hogImage)
    
# Converstion to Numpy Arrays
fds = np.asarray(fds)
hogs = np.asarray(hogs)

In [ ]:
# SIFT Algorithm (Scale Invariant Feature Transform) - Using OpenCV

# Function for finding the Keypoints and the Descriptors using SIFT
def siftDescriptor(image):
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)
    return kp, des

# For plotting the Keypoints
def drawkp(image, keypoints):
    img = cv2.drawKeypoints(np.uint8(image),
                     keypoints,
                     None,
                     flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return img

# Using the above two functions on the training dataset
kps, descs, kpimages = [], [], []
for i in range(len(train_X)):
    keypt, desc = siftDescriptor(train_X[i])
    kps.append(keypt)
    descs.append(desc)
    img = drawkp(train_X[i], keypt)
    kpimages.append(img)
    
# Conversion to Numpy Arrays
kps = np.asarray(kps, dtype='object')
descs = np.asarray(descs)
kpimages = np.asarray(kpimages)

In [ ]:
# Matching (using opencv) - Brute Force Matching (for SIFT based Descriptor)

# Function for the Brute Force Feature Matching using OpenCV
def bruteMatch(test_image, test_against_desc):
    matches, good = 0, []
    sift = cv2.SIFT_create()
    kpTest, desTest = siftDescriptor(test_image)
    bf = cv2.BFMatcher()
    if (desTest is not None) and (test_against_desc is not None):
        matches = bf.knnMatch(desTest, test_against_desc, k=2)
        for m, n in matches:
            if m.distance < 0.75*n.distance:
                good.append([m])
    return matches, good, kpTest

def bruteDraw(test_image, test_against_image, kp1, kp2, good):
    img = cv2.drawMatchesKnn(np.uint8(test_image),
                              kp1,
                              np.uint8(test_against_image),
                              kp2,
                              good,
                              None,
                              flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img

In [ ]:
# Feature Maatching using Brute Force Technique (OpenCV) - Best Match

# Test Image that needs to be Matched *(User-Input)*
test_image = test_X[25]

# Finding the best match - Scanning through the Training Dataset for the specifc test image
most_goods_len, most_goods_image_index, most_goods_matches, most_goods  = 0, 0, 0, 0
for i in range(len(train_X)):
    match, goods, kpTest = bruteMatch(test_image, descs[i])
    if match != 0 or len(goods) != 0 or kpTest != 0:
        if (len(goods) > most_goods_len):
            most_goods_len = len(goods)
            most_goods_image_index= i
            most_goods_matches = match
            most_goods = goods

# Feature Matching using the previously defined functions
best_image_match = bruteDraw(test_image, 
                             train_X[most_goods_image_index], 
                             kpTest, 
                             kps[most_goods_image_index], 
                             most_goods)

# Plotting the Best Matches 
if best_image_match is not None:
    person = ''
    classIndex = train.loc[most_goods_image_index, 'class']
    plt.imshow(best_image_match)
    plt.title("Feature Matching: Test Image (Left) | Best Match (Right)")
    if classIndex == 1:
        person = 'Jesse'
    elif classIndex == 2:
        person = 'Mila'
    else:
        person = 'Other'

    print(f"Closest Match --> Class: {classIndex}, Name: {person}")

In [ ]:
# Next steps
# code the HOG feature extractor for the train_X images, get the features
# do a matching using the feature matching also given in opencv and also check skimage
# then implement SIFT

### 1.1.1. t-SNE Plots
...

In [ ]:
# T-SNE Calculation using HOG FD
from sklearn.manifold import TSNE  
tsne = TSNE(n_components=2).fit_transform(fds)

In [ ]:
# t-SNE Plotting

# Scaling from 0 to 1 and starting from 0
def scaling(data):
    val_range = (np.max(data) - np.min(data))
    start0 = data - np.min(data)
    scaled = start0 / val_range 
    return scaled

# Scaling and Plotting the t-SNE
tx = tsne[:, 0]
ty = tsne[:, 1]
tx = scaling(tx)
ty = scaling(ty)
y_data = np.asarray(train.loc[:, 'class'])
plt.scatter(tx, ty, c=y_data, cmap=plt.cm.get_cmap("jet", 3))
plt.colorbar(ticks=range(3))
plt.show()

### 1.1.2. Discussion
...

## 1.2. Baseline 2: PCA feature extractor
...

In [ ]:
class PCAFeatureExtractor(IdentityFeatureExtractor):
    """TODO: this feature extractor is under construction"""
    
    def __init__(self, n_components):
        self.n_components = n_components
        
    def transform(self, X):
        raise NotImplmentedError
        
    def inverse_transform(self, X):
        raise NotImplmentedError

### 1.2.1. Eigenface Plots
...

### 1.2.2. Feature Space Plots
...

### 1.2.3. Discussion
...

# 2. Evaluation Metrics
## 2.0. Example: Accuracy
As example metric we take the accuracy. Informally, accuracy is the proportion of correct predictions over the total amount of predictions. It is used a lot in classification but it certainly has its disadvantages...

In [ ]:
from sklearn.metrics import accuracy_score

# 3. Classifiers
## 3.0. Example: The *'not so smart'* classifier
This random classifier is not very complicated. It makes predictions at random, based on the distribution obseved in the training set. **It thus assumes** that the class labels of the test set will be distributed similarly to the training set.

In [ ]:
class RandomClassificationModel:
    """Random classifier, draws a random sample based on class distribution observed 
    during training."""
    
    def fit(self, X, y):
        """Adjusts the class ratio instance variable to the one observed in y. 

        Parameters
        ----------
        X : tensor
            Training set
        y : array
            Training set labels

        Returns
        -------
        self : RandomClassificationModel
        """
        
        self.classes, self.class_ratio = np.unique(y, return_counts=True)
        self.class_ratio = self.class_ratio / self.class_ratio.sum()
        return self
        
    def predict(self, X):
        """Samples labels for the input data. 

        Parameters
        ----------
        X : tensor
            dataset
            
        Returns
        -------
        y_star : array
            'Predicted' labels
        """

        np.random.seed(0)
        return np.random.choice(self.classes, size = X.shape[0], p=self.class_ratio)
    
    def __call__(self, X):
        return self.predict(X)
    

## 3.1. Baseline 1: My favorite classifier
...

In [ ]:
class FavoriteClassificationModel:
    """TODO: this classifier is under construction."""
    
    def fit(self, X, y):
        raise NotImplmentedError
        
    def predict(self, X):
        raise NotImplmentedError

# 4. Experiments
<div class="alert alert-block alert-info"> <b>NOTE:</b> Do <i>NOT</i> use this section to keep track of every little change you make in your code! Instead, highlight the most important findings and the major (best) pipelines that you've discovered.  
</div>
<br>

## 4.0. Example: basic pipeline
The basic pipeline takes any input and samples a label based on the class label distribution of the training set. As expected the performance is very poor, predicting approximately 1/4 correctly on the training set. There is a lot of room for improvement but this is left to you ;). 

In [ ]:
feature_extractor = IdentityFeatureExtractor() 
classifier = RandomClassificationModel()

# train the model on the features
classifier.fit(feature_extractor(train_X), train_y)

# model/final pipeline
model = lambda X: classifier(feature_extractor(X))

In [ ]:
# evaluate performance of the model on the training set
train_y_star = model(train_X)

"The performance on the training set is {:.2f}. This however, does not tell us much about the actual performance (generalisability).".format(
    accuracy_score(train_y, train_y_star))

In [ ]:
# predict the labels for the test set 
test_y_star = model(test_X)

# 5. Publishing best results

In [ ]:
submission = test.copy().drop('img', axis = 1)
submission['class'] = test_y_star

submission

In [ ]:
submission.to_csv('submission.csv')

# 6. Discussion
...

In summary we contributed the following: 
* 
